In [1]:
import random

In [2]:
from faker import Factory
fake = Factory.create('ja_JP')

In [3]:
from flask import Flask
from flaskext.mysql import MySQL

In [4]:
app = Flask(__name__)
app.config.from_pyfile('../app/secret.cfg')

True

In [5]:
mysql = MySQL()
 
# MySQL configurations
app.config['MYSQL_DATABASE_USER'] = 'root'
app.config['MYSQL_DATABASE_DB'] = 'resas2017'
app.config['MYSQL_DATABASE_HOST'] = 'localhost'
mysql.init_app(app)

In [6]:
conn = mysql.connect()

In [7]:
cursor = conn.cursor()

In [8]:
cursor.execute('show tables;')

4

In [9]:
cursor.fetchall()

(('items',), ('keyword_relationships',), ('keywords',), ('stores',))

In [10]:
cursor.connection.user

b'root'

In [11]:
cursor.execute('select * from keywords;')
cursor.fetchall()

()

## Keywords

In [12]:
keyword_list = [
    '清水寺',
    '鹿苑寺',
    '伏見稲荷大社',
    '東映太秦映画村',
    '京都水族館',
    '北野天満宮',
    '貴船神社',
    '八坂神社',
    '三千院門跡',
    '慈照寺',
    '嵐山',
    '元離宮二条城',
    '平安神宮',
    '下鴨神社',
    '東寺'
]

In [13]:
len(keyword_list)

15

In [14]:
sql = 'insert into keywords(keyword) values("{}")'
for keyword in keyword_list:
    cursor.execute(sql.format(keyword))

In [15]:
cursor.execute('select count(1) from keywords;')
cursor.fetchall()

((15,),)

## Stores

In [16]:
column_number = 100

In [28]:
# 京都市の範囲
x1, y1 = 35.031280, 135.661159 
x2, y2 = 34.962638, 135.793442

In [17]:
for i in range(column_number):
    try:
        dummy_data = [
            # name
            fake.company(),

            # thumbnail
            fake.image_url(width=None, height=None),

            ## description
            fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None),

            # detail
            fake.text(max_nb_chars=200, ext_word_list=None),

            # lat
            str(random.uniform(x1,x2)),

            # lng
            str(random.uniform(y1,y2)),
        ]

        ret = '"'+'","'.join(dummy_data)+'"'
        name = dummy_data[0]

        sql = 'insert into stores(name, thumbnail, description, detail, lat, lng) values({})'
        cursor.execute(sql.format(ret))
    except:
        continue



In [18]:
cursor.execute('select id from stores;')
store_ids = [d[0] for d in cursor.fetchall()]

## relationships

In [21]:
# ひとつのStoreに対して，1〜3のkeywordを割り当てる

for i in store_ids:
    store_id = i
    for n in range(random.randint(a=1, b=3)):
        keyword_id = random.randint(a=1, b=len(keyword_list))
        cursor.execute('insert into keyword_relationships(keyword_id, store_id) values({}, {})'.format(keyword_id, store_id))

## items

In [22]:
item_number = 300

In [23]:
for i in range(item_number):
    store_id = random.choice(store_ids)
    url = fake.image_url(width=None, height=None)
    label = fake.word(ext_word_list=None)

    cursor.execute('insert into items(store_id, url, label) values({}, "{}", "{}")'.format(store_id, url, label))

In [24]:
conn.commit()

In [25]:
cursor.close()

In [26]:
conn.close()